# Урок 14. Transfer learning

взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
from corus import load_lenta2
path = 'lenta-ru-news.csv.bz2'

In [2]:
def load_lenta_to_list(path, max_number=None):
    records = load_lenta2(path)
    texts, titles = [], []
    for i, record in enumerate(records):
        texts.append(record.text)
        titles.append(record.title)
        if not max_number is None:
            if i >= max_number-1:
                break
    return texts, titles

In [3]:
import pandas as pd

In [4]:
texts, titles = load_lenta_to_list(path, max_number=15000)
df = pd.DataFrame({'text':texts, 'title':titles})
df

,text,title
0,Бои у Сопоцкина и Друскеник закончились отступ...,1914. Русские войска вступили в пределы Венгрии
1,"Министерство народного просвещения, в виду про...",1914. Празднование столетия М.Ю. Лермонтова от...
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",1914. Das ist Nesteroff!
3,Фотограф-корреспондент Daily Mirror рассказыва...,1914. Бульдог-гонец под Льежем
4,"Лица, приехавшие в Варшаву из Люблина, передаю...",1914. Под Люблином пойман швабский зверь
...,...,...
14995,На экстренной сессии Комиссии ООН по правам че...,Комиссия ООН расследует нарушения Израилем пра...
14996,Комиссия по организации аудиторской проверки г...,"ЗАО ""Делойт и Туш СНГ"" проведет аудит Центробанка"
14997,"Председатель Госдумы Геннадий Селезнев, входящ...",Геннадий Селезнев предал коммунистов
14998,Главнокомандующий Военно-Морским флотом России...,"Адмирал Куроедов прибыл на ""Регалию"" лично про..."


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dataset_train, dataset_test = train_test_split(df, test_size=0.1, random_state=1)

In [7]:
def len_tok(text):
    return len(text.split())

In [8]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['text'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(1111, 17)

In [9]:
max_len_sum, max_len_tl = 512, 15

In [10]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [11]:
from datasets import Dataset, DatasetDict

ds_data = DatasetDict({
    'train': Dataset.from_pandas(dataset_train),
    'test': Dataset.from_pandas(dataset_test)
})

dataset_train = ds_data['train']
dataset_test = ds_data['test']

In [12]:
dataset_train

Dataset({
    features: ['text', 'title', '__index_level_0__'],
    num_rows: 13500
})

In [13]:
dataset_test

Dataset({
    features: ['text', 'title', '__index_level_0__'],
    num_rows: 1500
})

In [14]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [15]:
def len_tok(text):
    return len(text.split())

In [16]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['text'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(1111, 17)

In [17]:
max_len_sum, max_len_tl = 512, 15

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

  0%|          | 0/1688 [00:00<?, ?ba/s]

  0%|          | 0/188 [00:00<?, ?ba/s]

In [19]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


model = T5ForConditionalGeneration.from_pretrained(model_name)

In [20]:
output_dir = 'output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [21]:
import torch
torch.cuda.empty_cache()

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\User\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13500
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 135000
  Number of trainable parameters = 244309248


Step,Training Loss,Validation Loss
500,4.439300,2.653798
1000,3.048800,2.268487
1500,2.638600,2.034596
2000,2.414700,1.923068
2500,2.211400,1.816076
3000,2.111700,1.693522
3500,2.178000,1.550574
4000,1.818700,1.546766
4500,1.803600,1.534660
5000,1.845600,1.508142


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-1000
Configuration saved in output\checkpoint-1000\config.json
Model weights saved in output\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument 

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-9000
Configuration saved in output\checkpoint-9000\config.json
Model weights saved in output\checkpoint-9000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-8000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.


***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-17000
Configuration saved in output\checkpoint-17000\config.json
Model weights saved in output\checkpoint-17000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-14000] due to args.save_total_limit
Deleting older checkpoint [output\checkpoint-16000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not ex

...

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-109000
Configuration saved in output\checkpoint-109000\config.json
Model weights saved in output\checkpoint-109000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-108000] due to args.save_tota

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-117000
Configuration saved in output\checkpoint-117000\config.json
Model weights saved in output\checkpoint-117000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-116000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this m

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-125000
Configuration saved in output\checkpoint-125000\config.json
Model weights saved in output\checkpoint-125000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-124000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this m

Model weights saved in output\checkpoint-133000\pytorch_model.bin
Deleting older checkpoint [output\checkpoint-132000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, __index_level_0__. If text, title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 2
Saving model checkpoint to output\checkpoint-134000
Configuration saved in output\checkpoint-134000\conf

TrainOutput(global_step=135000, training_loss=1.3176390032732928, metrics={'train_runtime': 38430.6892, 'train_samples_per_second': 7.026, 'train_steps_per_second': 3.513, 'total_flos': 1.835289280512e+17, 'train_loss': 1.3176390032732928, 'epoch': 20.0})

In [23]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to output/model
Configuration saved in output/model\config.json
Model weights saved in output/model\pytorch_model.bin


In [24]:
INX = 199
print("TEXT: | {}".format(dataset_test['text'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

TEXT: | Папа Римский Иоанн Павел II открыл в сочельник святые врата Главной католической базилики - собора Святого Петра, положив тем самым официальное начало празднованию 2000-летия Рождества Христова, сообщает Associated Press. Иоанн Павел II отслужил торжественную всенощную рождественскую службу. За церемонией наблюдали семь тысяч официальных гостей внутри собора и несколько десятков тысяч человек на Ватиканской площади, где были установлены гигантские мониторы. Святые врата церемониально открываются с 1300 года один раз в 25 лет. В субботу такие врата будут открыты во всех католических храмах мира. Перед вратами в новое тысячелетие человечество должно оставить все старое, грешное и облечься в новое и благодатное. "В Христе время и спасение встречаются в истории, которая тем самым становится историей спасения", - говорится в пасторском послании главы российских католиков к верующим. Он поздравил братьев-христиан, которые также празднуют сегодня Рождество, и православных, еще готовящ

In [34]:
import torch

device = "cuda"

for INX in range(50,60):
    print("TEXT: | {}".format(dataset_test['text'][INX]))
    print("TITLE: | {}".format(dataset_test['title'][INX]))

    input_text = dataset_test['text'][INX]

    with torch.no_grad():
        tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

        source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
        source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

        generated_ids = model.generate(
            input_ids = source_ids,
            attention_mask = source_mask, 
            max_length=512,
            num_beams=7,
            temperature = 1.3,
            repetition_penalty=1, 
            length_penalty=1, 
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    print("\noutput:\n" + pred)
    print(f'\n')
    print(100*'*')

TEXT: | Первые цистерны уже доставили бензин на особенно важные объекты Великобритании, однако пока Тони Блэр, пользуясь поддержкой королевы и правительства, пытается восстановить снабжение страны силовыми методами, опрос общественного мнения, проведенный BBC, показывает, что большая часть населения Англии поддерживает протестующих против высоких цен. По заверениям премьер-министра, в ближайшее время снабжение страны бензином будет восстановлено в полном объеме. Однако для беспрепятственной доставки бензина был необходим полицейский конвой, что вызывает у наблюдателей некоторые сомнения в обоснованности оптимистических прогнозов Тони Блэра. За текущие сутки митингующие в разных частях Англии пропустили несколько цистерн с бензином, предназначенных для объектов, особенно нуждающихся в топливе, - больниц, продуктовых магазинов, почтовых служб, однако волна протестов не пошла на спад. "Тони Блэр объявляет это своей победой, но он не улавливает реального состояния дел, - заявил представите


output:
В Петербурге ограблен менеджер компании "Тоннер"


****************************************************************************************************
TEXT: | Бундесканцлер Герхард Шредер обнародовал план правительства ФРГ по привлечению иностранных специалистов в области информационных технологий, разработанный в содружестве с немецкими промышленниками. Всего "для поднятия немецкой экономики" предполагается пригласить 20 тысяч специалистов в области информационных технологий, "при этом уже в июле их может приехать около 10 тысяч", пишет сегодняшняя Bild-Zeitung. В дальнейшем квота будет пересматриваться. В течение трех лет в отношении специалистов-компьютерщиков будет действовать облегченный визовый режим. Он предполагает для каждого гастарбайтера-компьютерщика занятость в Германии в течении пяти лет. Заявки на въезд будут рассматриваться только от лиц, имеющих высшее и специальное техническое образование. Оплата труда 20-тысячников будет вестись "по немецким стандартам", от